In [3]:
#check autoreload
from utils import *

In [ ]:
#---- download all agencias files from https://www.bcb.gov.br/fis/info/agencias.asp?frame=1
url = 'https://www.bcb.gov.br/fis/info/agencias.asp?frame=1' 
xpath = '/html/body/div/table/tbody/tr[2]/td[2]/form/select'
path = r'data'

for link in extract_download_links(url = url, xpath=xpath):
    complete_url = 'https://www.bcb.gov.br' + link
    print(complete_url)
    download_and_unzip(url = complete_url,extract_to=path)
    #? if it fails, toggle list reference
    print(f'year {complete_url[23:26]} and month {complete_url[27:29]} downloaded')

In [ ]:
#---- function to check col names for each file

# ! to give a bit of context: 
#the files col names have minor sintax changes across the years and months. 
#Also, the col names are not always in the same row.
#The function find_cnpj_row_number is supposed to find the row number of the first occurrence of the CNPJ value (which is always in the first column)
#and then skip all rows before that row.
#the function get_column_names is supposed to read all files in a folder, 
#extract the col names and return a dictionary with the file names as keys and the col names as values.
#It allows to see the pattern of col names across the multiple files

colnames_dict = get_column_names(path)
colnames_dict_count = count_list_values(colnames_dict)

In [5]:
#read each file and clean it 
dir_path = r'output'
path = r'data'


for file in os.listdir(path):
     # the files format change across the year
    if file.endswith('.xls') or file.endswith('.xlsx'):
        try:
            file_path = os.path.join(path, file)
            df = read_file(file_path=file_path, file_name=file)
            
            #create year and month objects to use in partioning
            ano = df['ano'][1] 
            mes = df['mes'][1]
        
            #general columns stardantization 
            df = clean_column_names(df)
            
            #rename columns
            df.rename(columns= rename_cols(), inplace= True)
            
            # fill left zeros with field range
            df['id_compe_bcb_agencia'] = df['id_compe_bcb_agencia'].astype(str).str.zfill(4)
            df['dv_do_cnpj'] = df['dv_do_cnpj'].astype(str).str.zfill(2)
            df['sequencial_cnpj'] = df['sequencial_cnpj'].astype(str).str.zfill(4)
            df['cnpj'] = df['cnpj'].astype(str).str.zfill(8)
            df['fone'] = df['fone'].astype(str).str.zfill(8)
            
            #check existence and create columns
            df = check_and_create_column(df, col_name='data_inicio')
            df = check_and_create_column(df, col_name='instituicao')
            df = check_and_create_column(df, col_name='id_instalacao')
            df = check_and_create_column(df, col_name='id_compe_bcb_instituicao')
            df = check_and_create_column(df, col_name='id_compe_bcb_agencia')

            #drop ddd column thats going to be added later
            df.drop(columns=['ddd'], inplace=True)
            print('ddd removed')
            print(df.columns)
            #some files doesnt have 'id_municipio', just 'nome'.
            #to add new ids is necessary to join by name
            #clean nome municipio
            df = clean_nome_municipio(df)
            
            
            municipio = read_brazilian_municipallity_ids_from_base_dos_dados('pisagab-staging')
            municipio = clean_nome_municipio(municipio)
            #check if id_municipio already exists
            
            if 'id_municipio' not in df.columns:
                #read municipio from bd datalake
                
                #join id_municipio to df
                df = pd.merge(df,
                municipio[['nome', 'sigla_uf', 'id_municipio','ddd']],
                left_on= ['nome','sigla_uf'],
                right_on = ['nome','sigla_uf'],
                how = 'left')

            #check if ddd already exists, if it doesnt, add it
            if 'ddd' not in df.columns:
                #
                #read municipio from bd datalake
                
                df = pd.merge(df,
                municipio[['id_municipio','ddd']],
                left_on= ['id_municipio'],
                right_on = ['id_municipio'],
                how = 'left')
            
            #clean cep column
            df['cep'] = df['cep'].astype(str)
            df['cep'] = df['cep'].apply(remove_non_numeric_chars)
            print('cep ok')

            # cnpj cleaning working
            df = create_cnpj_col(df)
            
            df['cnpj'] = df['cnpj'].apply(remove_non_numeric_chars)
            df['cnpj'] = df['cnpj'].apply(remove_empty_spaces)
            
            
            # select cols to title
            col_list_to_title = ['endereco', 'complemento' , 'bairro', 'nome_agencia']
            
            for col in col_list_to_title:
                str_to_title(df, column_name = col)
                print(f'column - {col} converted to title')

            #remove latin1 accents from all cols
            df = remove_latin1_accents_from_df(df)

            #format data_inicio
            df['data_inicio'] =  df['data_inicio'].apply(format_date) 
            
            #strip all df columns
            df = strip_dataframe_columns(df)
            
            #make sure month (mes) is an int to use as partition
            df['mes'] = df['mes'].astype(int)
            
            print(df.columns)
            print('ddd ordeder')
            df = df[order_cols()]

            #create partition
            partition = dir_path + f'/ano={ano}/mes={mes}/'
            
            #create dir if not exists
            if not os.path.exists(partition):
                os.makedirs(partition)
            print(f'dir ano_{ano}-mes_{mes} built')
        
            #add file name to partition
            partition = partition + 'agencia.csv'
            
            #final standartization of df
            df = replace_nan_with_empty_set_coltypes_str(df)

            df.to_csv(partition, index=False,  encoding='utf-8', na_rep='', sep = ',')
            
            print(f'df ano_{ano}-mes_{mes} saved')
        
            #keep track of possible erros that may occur
        except Exception as e:
            
            error = []
            error.append(f"Error ano{ano}-mes{mes}")
            print(f"Error ano{ano}-mes{mes}")
            print('error: ', e) 
    
#todo: some files have probls in columns 2021 3, 2020-06-01, 2019-12 e 4-1


8
200709AGENCIAS.xls
year 2007 and month 09 cols created
colnames striped
colnames cleaned 1
colnames cleaned 2
colnames to lower
ddd removed
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao',
       'id_compe_bcb_instituicao', 'segmento', 'id_compe_bcb_agencia',
       'nome_agencia', 'endereco', 'complemento', 'bairro', 'cep', 'nome',
       'sigla_uf', 'fone', 'ano', 'mes', 'data_inicio', 'id_instalacao'],
      dtype='object')


Downloading: 100%|██████████| 5570/5570 [00:01<00:00, 2828.87rows/s]


cep ok
cnpj built
column - endereco converted to title
column - complemento converted to title
column - bairro converted to title
column - nome_agencia converted to title
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao',
       'id_compe_bcb_instituicao', 'segmento', 'id_compe_bcb_agencia',
       'nome_agencia', 'endereco', 'complemento', 'bairro', 'cep', 'nome',
       'sigla_uf', 'fone', 'ano', 'mes', 'data_inicio', 'id_instalacao',
       'id_municipio', 'ddd'],
      dtype='object')
ddd ordeder
dir ano_2007-mes_09 built
df ano_2007-mes_09 saved
8
200710AGENCIAS.xls
year 2007 and month 10 cols created
colnames striped
colnames cleaned 1
colnames cleaned 2
colnames to lower
ddd removed
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao',
       'id_compe_bcb_instituicao', 'segmento', 'id_compe_bcb_agencia',
       'nome_agencia', 'endereco', 'complemento', 'bairro', 'cep', 'nome',
       'sigla_uf', 'fone', 'ano', 'mes', 'data_inicio', 'id_instalacao'],
      

Downloading: 100%|██████████| 5570/5570 [00:02<00:00, 2494.78rows/s]


cep ok
cnpj built
column - endereco converted to title
column - complemento converted to title
column - bairro converted to title
column - nome_agencia converted to title
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao',
       'id_compe_bcb_instituicao', 'segmento', 'id_compe_bcb_agencia',
       'nome_agencia', 'endereco', 'complemento', 'bairro', 'cep', 'nome',
       'sigla_uf', 'fone', 'ano', 'mes', 'data_inicio', 'id_instalacao',
       'id_municipio', 'ddd'],
      dtype='object')
ddd ordeder
dir ano_2007-mes_10 built
df ano_2007-mes_10 saved
8
200711AG.xls
year 2007 and month 11 cols created
colnames striped
colnames cleaned 1
colnames cleaned 2
colnames to lower
ddd removed
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao',
       'id_compe_bcb_instituicao', 'segmento', 'id_compe_bcb_agencia',
       'nome_agencia', 'endereco', 'complemento', 'bairro', 'cep', 'nome',
       'sigla_uf', 'fone', 'ano', 'mes', 'data_inicio', 'id_instalacao'],
      dtype=

...

cep ok
cnpj built
column - endereco converted to title
column - complemento converted to title
column - bairro converted to title
column - nome_agencia converted to title
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao', 'segmento',
       'id_compe_bcb_agencia', 'nome_agencia', 'endereco', 'numero',
       'complemento', 'bairro', 'cep', 'nome', 'sigla_uf', 'data_inicio',
       'fone', 'id_instalacao', 'id_municipio', 'ano', 'mes',
       'id_compe_bcb_instituicao', 'ddd'],
      dtype='object')
ddd ordeder
dir ano_2022-mes_12 built
df ano_2022-mes_12 saved
202301AGENCIAS.xlsx
year 2023 and month 01 cols created
colnames striped
colnames cleaned 1
colnames cleaned 2
colnames to lower
ddd removed
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao', 'segmento',
       'id_compe_bcb_agencia', 'nome_agencia', 'endereco', 'numero',
       'complemento', 'bairro', 'cep', 'nome', 'sigla_uf', 'data_inicio',
       'fone', 'id_instalacao', 'id_municipio', 'ano', 'mes',


Downloading: 100%|██████████| 5570/5570 [00:01<00:00, 2883.10rows/s]


cep ok
cnpj built
column - endereco converted to title
column - complemento converted to title
column - bairro converted to title
column - nome_agencia converted to title
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao', 'segmento',
       'id_compe_bcb_agencia', 'nome_agencia', 'endereco', 'numero',
       'complemento', 'bairro', 'cep', 'nome', 'sigla_uf', 'data_inicio',
       'fone', 'id_instalacao', 'id_municipio', 'ano', 'mes',
       'id_compe_bcb_instituicao', 'ddd'],
      dtype='object')
ddd ordeder
dir ano_2023-mes_01 built
df ano_2023-mes_01 saved
202302AGENCIAS.xlsx
year 2023 and month 02 cols created
colnames striped
colnames cleaned 1
colnames cleaned 2
colnames to lower
ddd removed
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao', 'segmento',
       'id_compe_bcb_agencia', 'nome_agencia', 'endereco', 'numero',
       'complemento', 'bairro', 'cep', 'nome', 'sigla_uf', 'data_inicio',
       'fone', 'id_instalacao', 'id_municipio', 'ano', 'mes',


Downloading: 100%|██████████| 5570/5570 [00:01<00:00, 3338.44rows/s]


cep ok
cnpj built
column - endereco converted to title
column - complemento converted to title
column - bairro converted to title
column - nome_agencia converted to title
Index(['cnpj', 'sequencial_cnpj', 'dv_do_cnpj', 'instituicao', 'segmento',
       'id_compe_bcb_agencia', 'nome_agencia', 'endereco', 'numero',
       'complemento', 'bairro', 'cep', 'nome', 'sigla_uf', 'data_inicio',
       'fone', 'id_instalacao', 'id_municipio', 'ano', 'mes',
       'id_compe_bcb_instituicao', 'ddd'],
      dtype='object')
ddd ordeder
dir ano_2023-mes_02 built
df ano_2023-mes_02 saved
